## Pytorch gradient test
Check how gradient calculation is working in mid-layers.
Below stream is,

$$x \xrightarrow{func(x)} z \xrightarrow{func2(z)} y$$

I want to know $$\frac{\partial y}{\partial z}$$, and $$\frac{\partial y}{\partial x}$$.

First, import torch and define input 1.

In [ ]:
import torch

x = torch.ones(1, requires_grad = True)

or do like,

In [ ]:
x = torch.ones(1)
x.requires_grad_(True)

Next, define `grad()`.
`tensor.register_hook(grad)` give the tensor's gradient to grad() as param.

In [ ]:
# save gradients.
def grad(grad):
    grads.append(grad)

Next, define $$func(x)$$ and $$func2(z)$$.

In [ ]:
def func(x):
    z = 2 * x
    z.register_hook(grad)
    return z

def func2(z):
    y = (z ** 2) + (2 * z) + 2
    y.register_hook(grad)
    return y

Next, feed x to $$func(x)$$ and $$func2(z)$$.

In [ ]:
z = func(x)
y = func2(z)

y.backward()

print(x.grad)